**Check for Java**

In [2]:
!java --version

java 16.0.1 2021-04-20
Java(TM) SE Runtime Environment (build 16.0.1+9-24)
Java HotSpot(TM) 64-Bit Server VM (build 16.0.1+9-24, mixed mode, sharing)


**Import and Check env for Tabula**

In [1]:
import tabula

In [2]:
tabula.environment_info()

Python version:
    3.8.5 (default, Jul 21 2020, 10:48:26) 
[Clang 11.0.3 (clang-1103.0.32.62)]
Java version:
    java version "16.0.1" 2021-04-20
Java(TM) SE Runtime Environment (build 16.0.1+9-24)
Java HotSpot(TM) 64-Bit Server VM (build 16.0.1+9-24, mixed mode, sharing)
tabula-py version: 2.2.0
platform: macOS-10.15.7-x86_64-i386-64bit
uname:
    uname_result(system='Darwin', node='Claytons-MBP.home', release='19.6.0', version='Darwin Kernel Version 19.6.0: Mon Apr 12 20:57:45 PDT 2021; root:xnu-6153.141.28.1~1/RELEASE_X86_64', machine='x86_64', processor='i386')
linux_distribution: ('Darwin', '19.6.0', '')
mac_ver: ('10.15.7', ('', '', ''), 'x86_64')
    


---------------

# What is Tabula?

"tabula-py is just a wrapper of tabula-java , make sure you’ve installed Java and you can use java command on your terminal. Many issue reporters forget to set PATH for java command.

You can check whether tabula-py can call java from Python process with tabula.environment_info() function." [FAQ](https://tabula-py.readthedocs.io/en/latest/faq.html)

In [159]:
pdf_path = '../data/Medical Surgical.pdf'
dfs = tabula.read_pdf(pdf_path, pages=3,lattice=True, stream=False, multiple_tables=True)

In [160]:
type(dfs)

list

In [161]:
len(dfs)

6

In [162]:
basicCareGauzeSponges = dfs[5]

In [163]:
basicCareGauzeSponges

,Sterile,Unnamed: 0,Unnamed: 1
0,8500,"2” x 2”, 8-ply","2/pk, 50 pk/bx, 30 bx/cs"
1,8501,"3” x 3”, 12-ply","2/pk, 50 pk/bx, 24 bx/cs"
2,8502,"4” x 4”, 8-ply","2/pk, 50 pk/bx, 12 bx/cs"
3,8503,"4” x 4”, 12-ply","2/pk, 25 pk/bx, 24 bx/cs"
4,Non-Sterile,NaN,NaN
5,8505,"2” x 2”, 8-ply","200/bg, 25 bg/cs"
6,8506,"2” x 2”, 12-ply","200/bg, 40 bg/cs"
7,8507,"3” x 3”, 8-ply","200/bg, 20 bg/cs"
8,8508,"3” x 3”, 12-ply","200/bg, 20 bg/cs"
9,8509,"4” x 4”, 8-ply","200/bg, 20 bg/cs"


In [164]:
new_column_names = ['Item_#', 'Decription', 'Packaging_Qty']
for col in basicCareGauzeSponges.columns:
    print(col)

Sterile
Unnamed: 0
Unnamed: 1


In [165]:
basicCareGauzeSponges.rename(columns={'Sterile': 'Item_#', 'Unnamed: 0': 'Description', 'Unnamed: 1': 'Packaging'}, inplace=True)


In [189]:
len(basicCareGauzeSponges)

11

In [187]:
sterile = []
for row in basicCareGauzeSponges.values:
    if row[0] == 'Non-Sterile':
        print(row[0])
        state = 'No'
        sterile.append(state)
        break
    else: 
        print(row[0])
        state = 'Yes'
        sterile.append(state)
    

    
    
    

8500
8501
8502
8503
Non-Sterile


In [188]:
sterile

['Yes', 'Yes', 'Yes', 'Yes', 'No']

In [152]:
basicCareGauzeSponges.insert(2, "Sterile", sterile)

In [168]:
for row in basicCareGauzeSponges.values:
    print(row[0])
        
        

8500
8501
8502
8503
Non-Sterile
8505
8506
8507
8508
8509
8510


In [128]:
basicCareGauzeSponges.dropna(inplace=True)

In [167]:
basicCareGauzeSponges

,Item_#,Description,Packaging
0,8500,"2” x 2”, 8-ply","2/pk, 50 pk/bx, 30 bx/cs"
1,8501,"3” x 3”, 12-ply","2/pk, 50 pk/bx, 24 bx/cs"
2,8502,"4” x 4”, 8-ply","2/pk, 50 pk/bx, 12 bx/cs"
3,8503,"4” x 4”, 12-ply","2/pk, 25 pk/bx, 24 bx/cs"
4,Non-Sterile,NaN,NaN
5,8505,"2” x 2”, 8-ply","200/bg, 25 bg/cs"
6,8506,"2” x 2”, 12-ply","200/bg, 40 bg/cs"
7,8507,"3” x 3”, 8-ply","200/bg, 20 bg/cs"
8,8508,"3” x 3”, 12-ply","200/bg, 20 bg/cs"
9,8509,"4” x 4”, 8-ply","200/bg, 20 bg/cs"


In [97]:
basicCareGauzeSponges.reset_index(drop=True, inplace=True)

In [98]:
basicCareGauzeSponges

,Item_#,Description,Sterile,Packaging
0,8500,"2” x 2”, 8-ply",Yes,"2/pk, 50 pk/bx, 30 bx/cs"
1,8501,"3” x 3”, 12-ply",Yes,"2/pk, 50 pk/bx, 24 bx/cs"
2,8502,"4” x 4”, 8-ply",Yes,"2/pk, 50 pk/bx, 12 bx/cs"
3,8503,"4” x 4”, 12-ply",Yes,"2/pk, 25 pk/bx, 24 bx/cs"
4,8505,"2” x 2”, 8-ply",No,"200/bg, 25 bg/cs"
5,8506,"2” x 2”, 12-ply",No,"200/bg, 40 bg/cs"
6,8507,"3” x 3”, 8-ply",No,"200/bg, 20 bg/cs"
7,8508,"3” x 3”, 12-ply",No,"200/bg, 20 bg/cs"
8,8509,"4” x 4”, 8-ply",No,"200/bg, 20 bg/cs"
9,8510,"4” x 4”, 12-ply",No,"200/bg, 10 bg/cs"


In [24]:
dfs.rename(columns={ df.columns[0]: "Left" , df.columns[1]: "Right",}, inplace = True)

[Empty DataFrame
 Columns: [Traditional Wound Care]
 Index: [],
 Empty DataFrame
 Columns: [Traditional Wound Care]
 Index: [],
         Sterile       Unnamed: 0                Unnamed: 1
 0         C5119   2” x 2”, 8-ply  2/pk, 50 pk/bx, 30 bx/cs
 1         C5129  3” x 3”, 12-ply  2/pk, 50 pk/bx, 24 bx/cs
 2         C5189   4” x 4”, 8-ply  2/pk, 50 pk/bx, 12 bx/cs
 3         C5139  4” x 4”, 12-ply  2/pk, 25 pk/bx, 24 bx/cs
 4          C517  8” x 4”, 12-ply  2/pk, 25 pk/bx, 20 bx/cs
 5   Non-Sterile              NaN                       NaN
 6        C82082   2” x 2”, 8-ply          200/bg, 25 bg/cs
 7        C82122  2” x 2”, 12-ply          200/bg, 40 bg/cs
 8        C83083   3” x 3”, 8-ply          200/bg, 20 bg/cs
 9        C83082  3” x 3”, 12-ply          200/bg, 20 bg/cs
 10       C84088   4” x 4”, 8-ply          200/bg, 20 bg/cs
 11       C84124  4” x 4”, 12-ply          200/bg, 10 bg/cs
 12       C84166  4” x 4”, 16-ply          200/bg, 10 bg/cs,
         Sterile        Unnamed

In [197]:
nums=[1,2,3,4,5,6,7,8,9]
for x in nums:
    while True:
        try:
            x > 6
        
        except:
            print('Except')
        
    

Except
Except
Except


KeyboardInterrupt: 